In [10]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [11]:
X = pd.read_csv("./data/X_train.csv")
Y = pd.DataFrame(X['Score'])
X = X.drop(columns=['Score'])
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [12]:
x_train_features = x_train.drop(columns=['Id', 'Text', 'Summary'])
x_test_features = x_test.drop(columns=['Id', 'Text', 'Summary'])

predictionSet = pd.read_csv("./data/prediction.csv")
x_predict = predictionSet.drop(columns=['Id', 'Text', 'Summary', 'Score'])

le = LabelEncoder()
x_train_features['ProductId'] = le.fit_transform(pd.DataFrame(x_train_features['ProductId']))
x_test_features['ProductId'] = le.fit_transform(pd.DataFrame(x_test_features['ProductId']))
x_predict['ProductId'] = le.fit_transform(pd.DataFrame(x_predict['ProductId']))
x_train_features['UserId'] = le.fit_transform(pd.DataFrame(x_train_features['UserId']))
x_test_features['UserId'] = le.fit_transform(pd.DataFrame(x_test_features['UserId']))
x_predict['UserId'] = le.fit_transform(pd.DataFrame(x_predict['UserId']))
x_train_features

,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time
1124647,40859,60711,1,1,1388016000
41595,757,31260,2,2,971654400
689600,21153,104071,0,0,1350086400
753431,24320,66389,7,12,1135728000
1080482,39169,37731,0,1,1318550400
...,...,...,...,...,...
93208,1381,37269,0,1,1210723200
289682,6725,38571,9,9,1217635200
905604,31905,28930,0,0,1378512000
166198,2886,67317,4,5,1059177600


In [13]:
# normalize all the columns
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

# x_train_features = normalize(x_train_features)
# x_test_features = normalize(x_test_features)
# x_predict = normalize(x_predict)
# x_train_features

In [14]:
model = GaussianNB().fit(x_train_features, y_train.values.flatten())
# model = KNeighborsClassifier(n_neighbors=3).fit(x_train_features, y_train.values.flatten())
predictionSet['Score'] = model.predict(x_predict)
x_test['Score'] = model.predict(x_test_features)
x_test = x_test.sort_values(by=['Id'])
x_test

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score
14,16,0005019281,A1GQF4JDJXBRLH,0,0,1173312000,Great Christmas movie for the family,This is a wonderful movie. It stay faithful to...,5.0
19,22,0005019281,A1PXS5N63PS6WR,2,3,1008115200,Change can be good,I must confess to being a bit of a coinsure of...,5.0
31,34,0005019281,AZB4CQ9JZSUQB,3,4,1012176000,A Christmas Carol to be remembered,"When this first aired in 1979, I enjoyed it so...",5.0
32,35,0005019281,A22GJ2SRENMZS0,0,0,1388966400,Dated TV Flic,Winkler does a little over-the-top in his role...,5.0
40,43,0005019281,A3L3ORBU5IXTMX,0,0,1388534400,An American Christmas Carol,I got this for a person who has it on vhs and ...,5.0
...,...,...,...,...,...,...,...,...,...
1397484,1697475,B00L4IDS4W,A2OOJSD9SG88RY,6,9,1404777600,So stupid! But awesome action.,"Don't worry, no spoilers here. Safe to read. G...",5.0
1397499,1697490,B00L8QP082,A1JA9HWFH51VJ9,0,1,1405814400,Thank You Universal!,30 classic horror films in one set... Excellen...,5.0
1397501,1697492,B00L8QP082,AXKA9U2AV1H6C,7,23,1404432000,One Star,No Blu-ray no sale wake up Universal !!!,5.0
1397516,1697509,B00LG7VVPO,ALND45MWBGUQS,25,31,1404604800,Looking at the True America,Let me start by saying that I don't agree with...,5.0


In [15]:
submission = predictionSet[['Id', 'Score']]
submission_offline = x_test[['Id', 'Score']]
print(submission.head())
submission.to_csv("./data/submission.csv", index=False)
submission_offline.to_csv("./data/submission_offline.csv", index=False)

   Id  Score
0   5    5.0
1  11    5.0
2  17    5.0
3  46    5.0
4  47    5.0
